In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>Credit Scoring Model – Logistic Regression (Client‑side)</title>
  <!-- TailwindCSS CDN -->
  <script src="https://cdn.tailwindcss.com"></script>
  <!-- Chart.js for ROC/PR charts -->
  <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
  <!-- PapaParse for CSV -->
  <script src="https://cdn.jsdelivr.net/npm/papaparse@5.4.1/papaparse.min.js"></script>
  <!-- TensorFlow.js for Logistic Regression -->
  <script src="https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@4.20.0/dist/tf.min.js"></script>
  <style>
    /* simple scrollbar + table tweaks */
    ::-webkit-scrollbar { height: 8px; width: 8px; }
    ::-webkit-scrollbar-thumb { background: #cbd5e1; border-radius: 8px; }
    .metric { @apply p-3 rounded-2xl shadow bg-white; }
    .card { @apply p-4 rounded-2xl shadow bg-white; }
    .badge { @apply text-xs px-2 py-1 rounded-full bg-slate-100; }
  </style>
</head>
<body class="bg-slate-50 text-slate-800">
  <div class="max-w-7xl mx-auto p-6 space-y-6">
    <header class="flex items-center justify-between">
      <div>
        <h1 class="text-2xl font-bold">Credit Scoring Model</h1>
        <p class="text-sm text-slate-500">Predict an individual's creditworthiness from past financial data – all in your browser.</p>
      </div>
      <div class="flex items-center gap-2">
        <button id="btnDemo" class="px-3 py-2 rounded-2xl bg-indigo-600 text-white hover:bg-indigo-700">Load Demo Data</button>
        <a href="#howto" class="px-3 py-2 rounded-2xl bg-slate-900 text-white hover:bg-black">How to Use</a>
      </div>
    </header>

    <!-- Data Loader -->
    <section class="card">
      <h2 class="text-lg font-semibold mb-3">1) Load Dataset (CSV)</h2>
      <div class="flex flex-wrap items-center gap-3">
        <input id="file" type="file" accept=".csv" class="border rounded-2xl p-2" />
        <span class="text-sm text-slate-500">CSV must have a header row. Numeric features only for v1 (categoricals can be pre‑encoded).</span>
      </div>
      <div id="dataPreview" class="mt-4 overflow-auto"></div>
    </section>

    <!-- Settings -->
    <section class="card">
      <h2 class="text-lg font-semibold mb-3">2) Configure</h2>
      <div class="grid md:grid-cols-3 gap-4">
        <div>
          <label class="text-sm font-medium">Target Column</label>
          <select id="targetCol" class="w-full border rounded-2xl p-2 mt-1"></select>
          <p class="text-xs text-slate-500 mt-1">Binary classification (0/1 or Yes/No). If text, set the positive class below.</p>
        </div>
        <div>
          <label class="text-sm font-medium">Positive Class Value</label>
          <input id="positiveVal" class="w-full border rounded-2xl p-2 mt-1" placeholder="e.g., 1 or 'Default'" />
          <p class="text-xs text-slate-500 mt-1">Rows matching this value are treated as positive (1). Others become 0.</p>
        </div>
        <div>
          <label class="text-sm font-medium">Algorithm</label>
          <select id="algo" class="w-full border rounded-2xl p-2 mt-1">
            <option value="logreg">Logistic Regression (tf.js)</option>
            <!-- Future: <option value="tree">Decision Tree</option> <option value="rf">Random Forest</option> -->
          </select>
          <p class="text-xs text-indigo-700 mt-1">This build includes Logistic Regression. Tree/Forest hooks are scaffolded for extension.</p>
        </div>
        <div>
          <label class="text-sm font-medium">Test Split: <span id="splitLabel">20%</span></label>
          <input id="split" type="range" min="10" max="50" step="5" value="20" class="w-full" />
        </div>
        <div>
          <label class="text-sm font-medium">Threshold: <span id="threshLabel">0.50</span></label>
          <input id="thresh" type="range" min="0" max="100" step="1" value="50" class="w-full" />
          <p class="text-xs text-slate-500 mt-1">Move to trade‑off Precision vs Recall. Metrics update live.</p>
        </div>
        <div class="flex items-end">
          <button id="btnTrain" class="w-full px-3 py-2 rounded-2xl bg-emerald-600 text-white hover:bg-emerald-700">3) Train Model</button>
        </div>
      </div>
    </section>

    <!-- Metrics -->
    <section class="grid md:grid-cols-3 gap-4">
      <div class="metric">
        <div class="text-xs uppercase text-slate-500">Accuracy</div>
        <div id="mAcc" class="text-2xl font-semibold">–</div>
      </div>
      <div class="metric">
        <div class="text-xs uppercase text-slate-500">Precision</div>
        <div id="mPrec" class="text-2xl font-semibold">–</div>
      </div>
      <div class="metric">
        <div class="text-xs uppercase text-slate-500">Recall</div>
        <div id="mRec" class="text-2xl font-semibold">–</div>
      </div>
      <div class="metric">
        <div class="text-xs uppercase text-slate-500">F1‑Score</div>
        <div id="mF1" class="text-2xl font-semibold">–</div>
      </div>
      <div class="metric">
        <div class="text-xs uppercase text-slate-500">ROC‑AUC</div>
        <div id="mAUC" class="text-2xl font-semibold">–</div>
      </div>
      <div class="metric">
        <div class="text-xs uppercase text-slate-500">Confusion Matrix</div>
        <table class="w-full text-sm mt-2">
          <thead>
            <tr><th></th><th class="text-center">Pred 0</th><th class="text-center">Pred 1</th></tr>
          </thead>
          <tbody>
            <tr><td class="font-medium">True 0</td><td id="cmTN" class="text-center">–</td><td id="cmFP" class="text-center">–</td></tr>
            <tr><td class="font-medium">True 1</td><td id="cmFN" class="text-center">–</td><td id="cmTP" class="text-center">–</td></tr>
          </tbody>
        </table>
      </div>
    </section>

    <!-- Charts -->
    <section class="grid md:grid-cols-2 gap-4">
      <div class="card">
        <div class="flex items-center justify-between mb-2">
          <h3 class="font-semibold">ROC Curve</h3>
          <span class="badge" id="rocPts">0 pts</span>
        </div>
        <canvas id="rocChart" height="280"></canvas>
      </div>
      <div class="card">
        <div class="flex items-center justify-between mb-2">
          <h3 class="font-semibold">Precision‑Recall Curve</h3>
          <span class="badge" id="prPts">0 pts</span>
        </div>
        <canvas id="prChart" height="280"></canvas>
      </div>
    </section>

    <!-- Feature Importance (LR Coefficients) -->
    <section class="card">
      <h2 class="text-lg font-semibold mb-3">Feature Importance (Logistic Regression Coefficients)</h2>
      <div id="featImp" class="overflow-auto"></div>
    </section>

    <!-- How to -->
    <section id="howto" class="card">
      <h2 class="text-lg font-semibold mb-3">How to Use</h2>
      <ol class="list-decimal ml-5 space-y-2 text-sm">
        <li>Click <span class="badge">Load Demo Data</span> or upload your own CSV with numeric columns (e.g., <code>income,debt,age,payment_history_score,target</code>).</li>
        <li>Choose the <b>Target Column</b> and set the <b>Positive Class</b> value (e.g., <code>1</code> for default).</li>
        <li>Pick the algorithm (Logistic Regression available), adjust <b>Test Split</b> and <b>Threshold</b>.</li>
        <li>Click <b>Train Model</b>. Metrics, charts, confusion matrix, and LR feature coefficients will appear.</li>
      </ol>
      <p class="text-xs text-slate-500 mt-2">All processing happens locally in your browser. No data leaves your machine.</p>
    </section>
  </div>

<script>
  // ===== Utilities =====
  const $ = (id) => document.getElementById(id);
  const format = (x, d=4) => (Number.isFinite(x) ? x.toFixed(d) : '–');

  function shuffleInPlace(arr) {
    for (let i = arr.length - 1; i > 0; i--) {
      const j = Math.floor(Math.random() * (i + 1));
      [arr[i], arr[j]] = [arr[j], arr[i]];
    }
    return arr;
  }

  function trainTestSplit(X, y, testPct=0.2) {
    const idx = [...Array(X.length).keys()];
    shuffleInPlace(idx);
    const nTest = Math.floor(X.length * testPct);
    const testIdx = idx.slice(0, nTest);
    const trainIdx = idx.slice(nTest);
    const Xtr = trainIdx.map(i => X[i]);
    const ytr = trainIdx.map(i => y[i]);
    const Xte = testIdx.map(i => X[i]);
    const yte = testIdx.map(i => y[i]);
    return { Xtr, ytr, Xte, yte };
  }

  function zscoreFit(X) {
    const n = X.length, d = X[0].length;
    const mean = Array(d).fill(0);
    const std = Array(d).fill(0);
    for (let i=0; i<n; i++) for (let j=0; j<d; j++) mean[j] += X[i][j];
    for (let j=0; j<d; j++) mean[j] /= n;
    for (let i=0; i<n; i++) for (let j=0; j<d; j++) std[j] += (X[i][j]-mean[j])**2;
    for (let j=0; j<d; j++) std[j] = Math.sqrt(std[j]/Math.max(1, n-1)) || 1;
    return { mean, std };
  }
  function zscoreTransform(X, stat) {
    const {mean, std} = stat; const n=X.length, d=X[0].length;
    const out = Array(n); for (let i=0; i<n; i++){ out[i]=Array(d); for(let j=0;j<d;j++){ out[i][j]=(X[i][j]-mean[j])/std[j]; }}
    return out;
  }

  // Metrics
  function confusion(yTrue, yPred) {
    let TP=0, TN=0, FP=0, FN=0;
    for (let i=0; i<yTrue.length; i++) {
      if (yTrue[i]===1 && yPred[i]===1) TP++;
      else if (yTrue[i]===0 && yPred[i]===0) TN++;
      else if (yTrue[i]===0 && yPred[i]===1) FP++;
      else if (yTrue[i]===1 && yPred[i]===0) FN++;
    }
    return {TP, TN, FP, FN};
  }
  function precisionRecallF1({TP,FP,FN}) {
    const precision = TP + FP === 0 ? 0 : TP / (TP + FP);
    const recall = TP + FN === 0 ? 0 : TP / (TP + FN);
    const f1 = (precision+recall)===0 ? 0 : (2*precision*recall)/(precision+recall);
    return { precision, recall, f1 };
  }
  function accuracy({TP,TN,FP,FN}) { return (TP+TN)/(TP+TN+FP+FN); }

  function rocCurve(yTrue, yScore) {
    // sort by descending score
    const pairs = yScore.map((s,i)=>[s,yTrue[i]]).sort((a,b)=>b[0]-a[0]);
    let P = yTrue.reduce((a,b)=>a+(b===1),0);
    let N = yTrue.length - P;
    let TP=0, FP=0;
    const pts = [[0,0]];
    for (let i=0; i<pairs.length; i++) {
      const [s, y] = pairs[i];
      if (y===1) TP++; else FP++;
      pts.push([FP/N, TP/P]);
    }
    pts.push([1,1]);
    // AUC via trapezoids
    let auc = 0; for (let i=1;i<pts.length;i++){ const [x0,y0]=pts[i-1], [x1,y1]=pts[i]; auc += (x1-x0)*(y0+y1)/2; }
    return { pts, auc };
  }

  function prCurve(yTrue, yScore) {
    const pairs = yScore.map((s,i)=>[s,yTrue[i]]).sort((a,b)=>b[0]-a[0]);
    let TP=0, FP=0; const P = yTrue.reduce((a,b)=>a+(b===1),0);
    const pts = [];
    for (let i=0;i<pairs.length;i++){
      const [s, y] = pairs[i];
      if (y===1) TP++; else FP++;
      const prec = TP/(TP+FP);
      const rec = TP/P;
      pts.push([rec, prec]);
    }
    return { pts };
  }

  // ===== Global State =====
  let DATA = { rows: [], cols: [], target: null, posVal: null, features: [], X: [], y: [] };
  let TRAIN = { Xtr:[], ytr:[], Xte:[], yte:[], scaler:null };
  let PRED = { proba: [], labels: [] };
  let MODEL = { tfModel: null, weights: null, featNames: [] };
  let rocChart, prChart;

  // ===== UI helpers =====
  function renderPreview(rows, maxRows=8) {
    if (!rows.length) { $('dataPreview').innerHTML = ''; return; }
    const cols = Object.keys(rows[0]);
    let html = '<div class="overflow-auto"><table class="min-w-full text-sm"><thead><tr>';
    html += cols.map(c=><th class="text-left p-2 bg-slate-100 sticky top-0">${c}</th>).join('');
    html += '</tr></thead><tbody>';
    rows.slice(0,maxRows).forEach(r=>{
      html += '<tr class="border-b">' + cols.map(c=><td class="p-2">${r[c]}</td>).join('') + '</tr>';
    });
    html += '</tbody></table></div>';
    $('dataPreview').innerHTML = html;
  }

  function populateTargetSelect(cols){
    const sel = $('targetCol'); sel.innerHTML = '';
    cols.forEach(c=>{ const opt=document.createElement('option'); opt.value=c; opt.textContent=c; sel.appendChild(opt); });
  }

  function updateMetricsDisplay(conf, prf, acc, auc){
    $('mAcc').textContent = format(acc,4);
    $('mPrec').textContent = format(prf.precision,4);
    $('mRec').textContent = format(prf.recall,4);
    $('mF1').textContent = format(prf.f1,4);
    $('mAUC').textContent = format(auc,4);
    $('cmTP').textContent = conf.TP; $('cmTN').textContent = conf.TN; $('cmFP').textContent = conf.FP; $('cmFN').textContent = conf.FN;
  }

  function drawChart(ctx, label, xy){
    const data = {
      labels: xy.map(p=>p[0]),
      datasets: [{ label, data: xy.map(p=>({x:p[0], y:p[1]})), pointRadius: 0 }]
    };
    const cfg = { type: 'scatter', data,
      options: { responsive: true, maintainAspectRatio: false, parsing: false,
        scales: { x: { type: 'linear', min: 0, max: 1 }, y: { type: 'linear', min: 0, max: 1 } },
        plugins: { legend: { display: false } }
      }
    };
    return new Chart(ctx, cfg);
  }

  function updateCharts(rocPts, prPts){
    $('rocPts').textContent = ${rocPts.length} pts;
    $('prPts').textContent = ${prPts.length} pts;
    if (rocChart) rocChart.destroy();
    if (prChart) prChart.destroy();
    rocChart = drawChart($('rocChart').getContext('2d'), 'ROC', rocPts);
    prChart = drawChart($('prChart').getContext('2d'), 'PR', prPts);
  }

  function renderFeatureImportance(names, weights){
    if (!weights || !weights.length) { $('featImp').innerHTML = '<div class="text-sm text-slate-500">Train a Logistic Regression model to view coefficients.</div>'; return; }
    const absw = weights.map(Math.abs);
    const maxAbs = Math.max(...absw) || 1;
    let html = '<table class="min-w-full text-sm"><thead><tr><th class="text-left p-2 bg-slate-100">Feature</th><th class="text-left p-2 bg-slate-100">Weight</th><th class="text-left p-2 bg-slate-100">|Weight|</th></tr></thead><tbody>';
    names.forEach((f,i)=>{
      const barWidth = Math.round((absw[i]/maxAbs)*100);
      html += <tr class="border-b"><td class="p-2">${f}</td><td class="p-2">${weights[i].toFixed(6)}</td><td class="p-2"><div class="bg-slate-200 h-2 rounded-full"><div class="h-2 rounded-full" style="width:${barWidth}%"></div></div></td></tr>;
    });
    html += '</tbody></table>';
    $('featImp').innerHTML = html;
  }

  // ===== Data Handling =====
  function inferNumericCols(rows, target){
    if (!rows.length) return [];
    const cols = Object.keys(rows[0]).filter(c=>c!==target);
    const numeric = [];
    for (const c of cols){
      let ok=true;
      for (let i=0;i<rows.length;i++){
        const v = parseFloat(rows[i][c]);
        if (Number.isNaN(v)) { ok=false; break; }
      }
      if (ok) numeric.push(c);
    }
    return numeric;
  }

  function buildXY(rows, target, positiveVal){
    const featCols = inferNumericCols(rows, target);
    const X=[]; const y=[];
    for (const r of rows){
      const rowX = featCols.map(c=>parseFloat(r[c]));
      if (rowX.some(v=>Number.isNaN(v))) continue; // skip bad row
      let label = r[target];
      if (positiveVal!==null && positiveVal!=="" && r[target]!==undefined){
        label = (String(r[target]).trim() === String(positiveVal).trim()) ? 1 : 0;
      } else {
        label = Number(r[target]);
      }
      if (label!==0 && label!==1) continue; // enforce binary
      X.push(rowX); y.push(label);
    }
    return { X, y, featCols };
  }

  // ===== Model: Logistic Regression via tf.js =====
  async function trainLogReg(Xtr, ytr, epochs=200, lr=0.05){
    const nFeat = Xtr[0].length;
    const model = tf.sequential();
    model.add(tf.layers.dense({ units: 1, inputShape: [nFeat], activation: 'sigmoid', useBias: true }));
    const opt = tf.train.adam(lr);
    model.compile({ optimizer: opt, loss: 'binaryCrossentropy' });
    const x = tf.tensor2d(Xtr);
    const y = tf.tensor2d(ytr, [ytr.length,1]);
    await model.fit(x, y, { epochs, batchSize: Math.min(64, Xtr.length), verbose: 0 });
    x.dispose(); y.dispose();
    return model;
  }

  async function predictProba(model, X){
    const x = tf.tensor2d(X);
    const p = model.predict(x);
    const proba = Array.from(await p.data());
    tf.dispose([x,p]);
    return proba;
  }

  function extractWeights(model){
    const ws = model.getWeights();
    if (!ws.length) return null;
    // [kernel, bias]
    const kernel = ws[0];
    const bias = ws[1];
    return tf.tidy(()=>{
      const k = kernel.arraySync().map(row=>row[0]);
      const b = bias.arraySync()[0];
      return { weights: k, bias: b };
    });
  }

  // ===== Orchestration =====
  function recomputeMetricsAtThreshold(thresh){
    const yTrue = TRAIN.yte;
    const yHat = PRED.proba.map(p => p>=thresh ? 1 : 0);
    PRED.labels = yHat;
    const conf = confusion(yTrue, yHat);
    const prf = precisionRecallF1(conf);
    const acc = accuracy(conf);
    const roc = rocCurve(yTrue, PRED.proba);
    const pr = prCurve(yTrue, PRED.proba);
    updateMetricsDisplay(conf, prf, acc, roc.auc);
    updateCharts(roc.pts, pr.pts);
  }

  // ===== Event Listeners =====
  $('split').addEventListener('input', e=>{ $('splitLabel').textContent = e.target.value + '%'; });
  $('thresh').addEventListener('input', e=>{ const t=Number(e.target.value)/100; $('threshLabel').textContent=t.toFixed(2); if (PRED.proba.length) recomputeMetricsAtThreshold(t); });

  $('file').addEventListener('change', (e)=>{
    const f = e.target.files[0]; if (!f) return;
    Papa.parse(f, { header: true, skipEmptyLines: true, complete: (res)=>{
      DATA.rows = res.data;
      DATA.cols = res.meta.fields || Object.keys(DATA.rows[0]||{});
      renderPreview(DATA.rows);
      populateTargetSelect(DATA.cols);
    }});
  });

  $('btnDemo').addEventListener('click', ()=>{
    // Small synthetic demo: default = 1 if high debt-to-income and poor payment score
    const csv = `income,debt,age,payment_score,default\n
    65000,12000,34,0.82,0\n
    42000,18000,29,0.35,1\n
    75000,15000,41,0.78,0\n
    30000,22000,26,0.22,1\n
    52000,8000,31,0.71,0\n
    41000,20000,27,0.28,1\n
    90000,10000,45,0.85,0\n
    28000,24000,24,0.18,1\n
    60000,9000,36,0.76,0\n
    35000,21000,25,0.25,1\n
    68000,11000,39,0.80,0\n
    50000,17000,33,0.40,1\n
    72000,7000,38,0.83,0\n
    32000,23000,28,0.20,1\n
    58000,9500,35,0.74,0\n
    40000,19000,27,0.30,1`;
    const res = Papa.parse(csv.trim(), { header: true, skipEmptyLines: true });
    DATA.rows = res.data;
    DATA.cols = res.meta.fields;
    renderPreview(DATA.rows);
    populateTargetSelect(DATA.cols);
    $('targetCol').value = 'default';
    $('positiveVal').value = '1';
  });

  $('btnTrain').addEventListener('click', async ()=>{
    if (!DATA.rows.length){ alert('Please upload a CSV or load the demo data.'); return; }
    const target = $('targetCol').value; if (!target){ alert('Choose a target column'); return; }
    const pos = $('positiveVal').value;

    const { X, y, featCols } = buildXY(DATA.rows, target, pos);
    if (X.length < 10) { alert('Need at least 10 valid rows after cleaning.'); return; }

    DATA.target = target; DATA.posVal = pos; DATA.features = featCols; DATA.X = X; DATA.y = y;

    const testPct = Number($('split').value)/100;
    const { Xtr, ytr, Xte, yte } = trainTestSplit(X, y, testPct);
    const scaler = zscoreFit(Xtr);
    const XtrS = zscoreTransform(Xtr, scaler);
    const XteS = zscoreTransform(Xte, scaler);
    TRAIN = { Xtr: XtrS, ytr, Xte: XteS, yte, scaler };

    // Algorithm switch (currently only logreg)
    const algo = $('algo').value;
    if (algo !== 'logreg') { alert('This build implements Logistic Regression.'); return; }

    // Train
    $('btnTrain').disabled = true; $('btnTrain').textContent = 'Training…';
    const model = await trainLogReg(TRAIN.Xtr, TRAIN.ytr, 300, 0.05);
    $('btnTrain').disabled = false; $('btnTrain').textContent = 'Re‑Train Model';

    // Predict
    const proba = await predictProba(model, TRAIN.Xte);
    PRED.proba = proba;

    // Metrics at current threshold
    const thresh = Number($('thresh').value)/100; $('threshLabel').textContent = thresh.toFixed(2);
    recomputeMetricsAtThreshold(thresh);

    // Coefficients
    const w = extractWeights(model);
    MODEL = { tfModel: model, weights: w?.weights || [], bias: w?.bias || 0, featNames: DATA.features };
    renderFeatureImportance(MODEL.featNames, MODEL.weights || []);
  });

  // Init selects empty
  populateTargetSelect([]);
  renderFeatureImportance([], []);
</script>
</body>
</html>